In [ ]:
import pyupbit
import time
import requests
import json
import datetime
import math
import schedule

#######################################################
##글로벌 변수 선언 ####################################
#######################################################
access_key = "USE YOUR ACCESS KEY"
secret_key = "USE YOUR SECRET KEY"
upbit = pyupbit.Upbit(access_key, secret_key)
url = "https://api.alternative.me/fng/?limit="
########################################################



########################################################
#### fear_day() : 그날의 공포 탐욕 지수를 불러오는 함수#
########################################################
def fear_day():
    _url = url+"1"
    res = requests.request("GET", _url);
    
    parsed = json.loads(res.text)
    data = parsed["data"]
    
    return data[0]["value"]

########################################################
#### my_krw_asset() : KRW 금액 구하기 ##################
########################################################
def my_krw_asset():
    KRW = 0
    for i in range(len(upbit.get_balances())):
        if upbit.get_balances()[i]['currency'] == "KRW":
                   KRW = upbit.get_balances()[i]['balance']
    
    return KRW

########################################################
#### my_asset() : KRW, BTC의 총 자산합계 구하기 ########
########################################################
def my_asset():
    KRW = my_krw_asset()
    BTC = 0

    for i in range(len(upbit.get_balances())):
        if upbit.get_balances()[i]['currency'] == "BTC":
                   BTC = upbit.get_balances()[i]['balance']
           
    print("보유 KRW = ",KRW)
    BTCKRW = float(BTC) * pyupbit.get_current_price("KRW-BTC")
    print("보유 BTC (KRW 환산)",BTCKRW )
    TOTAL = float(KRW) + float(BTCKRW)
    print("총 자산 = ", TOTAL)
    
    return TOTAL

def bitcoin():
    # KRW 불러오기
    KRW = my_krw_asset()
    # 나의 총자산 불러오기
    TOTAL = my_asset()
    # 공포지수 불러오기
    FEAR = fear_day()
    print("공포지수 : ",FEAR)
    
    # CASH: 현금비율, 현금비율 = 공포지수로 했을 시, 필요한 현금 보유량 계산
    CASH = int(TOTAL * float(FEAR) / 100) 
    print("CASH = ", CASH)
    
    # 현재 가지고 있는 KRW와 공포지수의 차이
    DIFF = int(float(KRW)) - CASH
    print("차이 = ",DIFF)
    
    # 매수,매도 주문금액이 5000보다 작으면 return (업비트 최소주문금액 : 5000원 )
    if DIFF in range(-5000,5000):
        print("최소주문금액보다 작음[ ",DIFF," ]")
        return
    
    # 매수할 것인지 매도할 것인지 체크
    if DIFF > 0 :
        MESUMEDO = 0 ## 현금보유비중이 공포지수보다 낮은 경우 0 : 매수
        print("매수실행")
    elif DIFF < 0:
        MESUMEDO = 1 ## 현금보유비중이 공포지수보다 높은 경우 1 : 매도
        DIFF = DIFF * -1
        print("매도실행")
 
    #### 매매 ####
    orderbook = pyupbit.get_orderbook("KRW-BTC")
    
    if MESUMEDO == 0 : # 매수
        # 매도 호가로 매수
        buy = orderbook['orderbook_units'][0]['ask_price'] 
        count = round(DIFF/buy,8)
        print("매수금액 = ",count*buy)
        ret = upbit.buy_limit_order("KRW-BTC",buy,count)
    elif MESUMEDO == 1 : # 매도
        # 매수 호가로 매도
        sell = orderbook['orderbook_units'][0]['bid_price']
        count = round(DIFF/sell,8)
        print("매도금액 = ",count*sell)
        ret = upbit.sell_limit_order("KRW-BTC",sell,count)

while True:
    today = datetime.datetime.today()
    print(today, "...........대기중")
    if today.hour == 9 and today.minute in range(0,60):
        print("시작!")
        bitcoin()
        print("종료!")
        time.sleep(3600)
    elif today.hour == 8:
        time.sleep(60)
    else:
        time.sleep(3600)

2021-12-17 23:58:41.025673 ...........대기중
시작!
보유 KRW =  232026.32297844
보유 KRW =  232026.32297844
보유 KRW =  232026.32297844
보유 BTC (KRW 환산) 752952.30435
총 자산 =  984978.62732844
공포지수 :  23
CASH =  226545
차이 =  5481
매수실행
매수금액 =  5480.98128
종료!
